# Folium 라이브러리 - 지도 시각화

- Folium 라이브러리는 지도 위에 시각화할 때 유용한 도구이다.
- 세계 지도를 기본 지원하고 다양한 스타일의 지도 이미지를 제공한다.   

https://python-visualization.github.io/folium/index.html

### Folium 설치

아나콘다 프롬프트에서 ***conda install -c conda-forge folium*** 을 입력한다.

## 지도 만들기

- Map() 함수를 이용하면 간단하게 지도 객체를 만들수 있다.
- 지도 화면은 고정된 것이 아니고 zoom 기능과 화면 이동( scroll )이 모두 가능하다.   


- Folium은 웹 기반 지도를 만들어 준다. 오직 웹 환경에서만 지도를 확인할 수 있다.
- 웹 환경으로 지도를 볼 수 없는 경우에는 save() 메서드를 적용하여 HTML 파일로 저장하고, 웹브라우저에서 파일을 열어 확인한다.
- jupyter notebook등 웹 기반 IDE에서는 지도 객체를 바로 확인할 수 있다.   

- Map() 함수의 location 옵션에 [ 위도, 경도 ] 수치를 입력하면 그 지점을 중심으로 지도를 보여준다.
- zoom_start 옵션을 사용하면 화면 확대 비율을 조절할 수 있다.

In [1]:
import pandas as pd
import folium

import json

In [2]:
# 서울 지도 만들기
seoul_map = folium.Map(location=[37.55,126.98], zoom_start=12)

# 지도를 HTML 파일로 저장하기
seoul_map.save('./seoul.html')

In [3]:
seoul_map

## 지도 스타일 적용하기

- Map() 함수에 tiles 옵션을 적용하면 지도에 적용하는 스타일을 변경하여 지정할 수 있다.

In [4]:
# 서울 지도 만들기
seoul_map2 = folium.Map(location=[37.55,126.98], tiles='Stamen Terrain', 
                        zoom_start=12)  # 산악 지형 맵

seoul_map3 = folium.Map(location=[37.55,126.98], tiles='Stamen Toner', 
                        zoom_start=15) # 흑백 스타일로 도로망 맵

In [5]:
seoul_map2

In [6]:
seoul_map3

## 지도에 마커 표시

- 마커 위치를 표시하려면 Marker() 함수에 위도, 경도 정보를 전달한다.
- popup 옵션을 추가하면 마커를 클릭했을 때 팝업창에 표시해주는 텍스트를 넣을 수 있다.
- 원형 마커를 표시할 때는 CircleMarker() 함수를 사용한다. 원형 마커의 크기, 색상, 투명도 등을 설정할 수 있다.

In [15]:
# 대학교 리스트를 데이터프레임 변환
df = pd.read_excel('./서울지역 대학교 위치.xlsx', encoding = 'utf-8' )

In [16]:
# 서울 지도 만들기
seoul_map = folium.Map(location=[37.55,126.98], tiles='Stamen Terrain', 
                        zoom_start=12)

# 대학교 위치정보를 Marker로 표시
for name, lat, lng in zip(df.학교명, df.위도, df.경도):
    folium.Marker([lat, lng], popup=name).add_to(seoul_map)

seoul_map
# 지도를 HTML 파일로 저장하기
seoul_map.save('./seoul_colleges.html')

In [11]:
# 서울 지도 만들기
seoul_map = folium.Map(location=[37.55,126.98], tiles='Stamen Terrain', 
                        zoom_start=12)

# 대학교 위치정보를 CircleMarker로 표시
for name, lat, lng in zip(df.학교명, df.위도, df.경도):
    folium.CircleMarker([lat, lng],
                        radius=10,         # 원의 반지름
                        color='brown',         # 원의 둘레 색상
                        fill=True,
                        fill_color='coral',    # 원을 채우는 색
                        fill_opacity=0.7, # 투명도    
                        popup=name
    ).add_to(seoul_map)

seoul_map
   # 지도를 HTML 파일로 저장하기
#seoul_map.save('./seoul_colleges2.html')

## 지도 영역에 단계구분도( Choropleth Map ) 표시

- 행정구역과 같이 지도 상의 어떤 경계에 둘러싸인 영역에 색을 칠하거나 음영 등으로 정보를 나타내는 시각화 방법이다.
- 전달하려는 정보의 값이 커지면 영역에 칠해진 색이나 음영이 진해진다.
- Choropleth() 함수를 이용한다.

folium.Choropleth(   
       geo_data = "지도 데이터 파일 경로 (.geojson, geopandas.DataFrame)"   
       data = "시각화 하고자 하는 데이터파일. (pandas.DataFrame)"   
       columns = (지도 데이터와 매핑할 값, 시각화 하고자하는 변수),   
       key_on = "feature.데이터 파일과 매핑할 값",   
       fill_color = "시각화에 쓰일 색상", legend_name = "칼라 범주 이름",   
).add_to(m)   

출처: https://dailyheumsi.tistory.com/144

In [13]:
# 경기도 인구변화 데이터를 불러와서 데이터프레임으로 변환
file_path = './경기도인구데이터.xlsx'
df = pd.read_excel(file_path, index_col='구분')  
df.columns = df.columns.map(str)

# 경기도 시군구 경계 정보를 가진 geo-json 파일 불러오기
geo_path = './경기도행정구역경계.json'
try:
    geo_data = json.load(open(geo_path, encoding='utf-8'))
except:
    geo_data = json.load(open(geo_path, encoding='utf-8-sig'))

# 경기도 지도 만들기
g_map = folium.Map(location=[37.5502,126.982], 
                   tiles='Stamen Terrain', zoom_start=9)

# 출력할 연도 선택 (2007 ~ 2017년 중에서 선택)
year = '2007'  

# Choropleth 클래스로 단계구분도 표시하기
folium.Choropleth(geo_data=geo_data,    # 지도 경계
                 data = df[year],      # 표시하려는 데이터
                 columns = [df.index, df[year]],  # 열 지정
                 fill_color='YlOrRd', fill_opacity=0.7, line_opacity=0.3,
                 threshold_scale=[10000, 100000, 300000, 500000, 700000],               
                 key_on='feature.properties.name',
                 ).add_to(g_map)
g_map

# 지도를 HTML 파일로 저장하기
#g_map.save('./gyonggi_population_' + year + '.html')